In [4]:
import pandas as pd
import pandas_profiling
import os
import pickle
import gc
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns


import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import Lasso
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold



#評価用

train = pd.read_csv("data_EDA/train.csv")
test = pd.read_csv("data_EDA/test.csv")

with open('ensamble/rf_train.pickle', mode='br') as fi:
    train_p1 = pickle.load(fi)
    
with open('ensamble/lgb_train.pickle', mode='br') as fi:
    train_p2 = pickle.load(fi)
    
with open('ensamble/tabnet_w_train.pickle', mode='br') as fi:
    train_p3 = pickle.load(fi)
    
with open('ensamble/nnw_train.pickle', mode='br') as fi:
    train_p4 = pickle.load(fi)
    
with open('ensamble/lgr_train.pickle', mode='br') as fi:
    train_p5 = pickle.load(fi)






#データ読み込み
with open('ensamble/rf_test.pickle', mode='br') as fi:
    test_p1 = pickle.load(fi)
    
with open('ensamble/lgb_test.pickle', mode='br') as fi:
    test_p2 = pickle.load(fi)
    
with open('ensamble/tabnet_w_test.pickle', mode='br') as fi:
    test_p3 = pickle.load(fi)
    
with open('ensamble/nnw_test.pickle', mode='br') as fi:
    test_p4 = pickle.load(fi)
    
with open('ensamble/lgr_test.pickle', mode='br') as fi:
    test_p5 = pickle.load(fi)
    

## アンサンブル

In [15]:
#　評価用
df_train = pd.concat(
    [train['Outcome'],train_p1['pred'],train_p2['pred'], train_p3['pred'], train_p4['pred'], train_p5['pred']], 
                     axis=1, ignore_index=True).rename(columns={
    0: 'true', 1: 'pred1', 2: 'pred2', 3:'pred3', 4:'pred4',5:'pred5'
})

df_train['ensemble'] = df_train['pred1']*0.4 + df_train['pred2']*0.5 +df_train['pred3']*0.1 #+ df_train['pred4']*0.1 +  df_train['pred5']*0.1

# 提出用
df_test = pd.concat(
    [test['index'],test_p1['pred'],test_p2['pred'], test_p3['pred'],  test_p4['pred'], test_p5['pred']], 
                    axis=1, ignore_index=True).rename(columns={
    0: 'index', 1: 'pred1', 2: 'pred2', 3:'pred3', 4:'pred4', 5:'pred5'
})

df_test['ensemble'] = df_test['pred1']*0.4 + df_test['pred2']*0.5  +df_test['pred3']*0.1 #+ df_test['pred4']*0.1 + df_test['pred5']*0.1


def evaluate_ensemble(input_df, col_pred):
    print('[auc] model1:{:.4f},model2:{:.4f},model3:{:.4f},model4:{:.4f},model5:{:.4f}  ->    emsamble]{:.4f}'.format(
        roc_auc_score(input_df['true'], input_df['pred1']),
        roc_auc_score(input_df['true'], input_df['pred2']),
        roc_auc_score(input_df['true'], input_df['pred3']),
        roc_auc_score(input_df['true'], input_df['pred4']),
        roc_auc_score(input_df['true'], input_df['pred5']),
        
        
        roc_auc_score(input_df['true'], input_df[col_pred]),
    ))

In [16]:
df_train.head(20)

,true,pred1,pred2,pred3,pred4,pred5,ensemble
0,1.0,0.442568,0.589304,0.663979,0.398457,0.402233,0.538077
1,0.0,0.057432,0.057239,0.080110,0.114412,0.073798,0.059603
2,0.0,0.530405,0.354875,0.792315,0.413883,0.412125,0.468831
3,0.0,0.628378,0.615251,0.982137,0.499567,0.602159,0.657190
4,1.0,0.847973,0.527615,0.993841,0.544948,0.689416,0.702381
5,0.0,0.070946,0.079734,0.287218,0.118313,0.173991,0.096967
6,1.0,0.060811,0.066999,0.186049,0.158183,0.124462,0.076429
7,1.0,0.483108,0.313623,0.889087,0.427832,0.519163,0.438963
8,0.0,0.141892,0.077855,0.284754,0.181301,0.202901,0.124159
9,0.0,0.334459,0.314537,0.856990,0.414141,0.347957,0.376751


In [17]:
evaluate_ensemble(df_train, col_pred='ensemble')

[auc] model1:0.8061,model2:0.8215,model3:0.7669,model4:0.7536,model5:0.7537  ->    emsamble]0.8216


In [18]:
sub = df_test.copy()    
sub['ensemble'] = np.where(sub['ensemble'] < 0.5, 0, 1)
sub = sub[['index', 'ensemble']] 
sub.to_csv('sub/submission_rf+lgb+tab.csv', index=None, header=False,)

## スタッキング

In [19]:
x, y = df_train[['pred1', 'pred2', ]], df_train[['true']]
oof = np.zeros(len(x))
models = []

cv = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=123).split(x,y))
for nfold in np.arange(5):
    idx_tr, idx_va = cv[nfold][0], cv[nfold][1]
    x_tr, y_tr = x.loc[idx_tr, :], y.loc[idx_tr, :]
    x_va, y_va = x.loc[idx_va, :], y.loc[idx_tr, :]
    
    model = Lasso(alpha=0.01)
    model.fit(x_tr, y_tr)
    models.append(model)
    
    y_va_pred = model.predict(x_va)
    oof[idx_va] = y_va_pred
    
df_train['stacking'] = oof
df_train['stacking'] = df_train['stacking'].clip(lower=0, upper=1)
df_train.head(20)

,true,pred1,pred2,pred3,pred4,pred5,ensemble,stacking
0,1.0,0.442568,0.589304,0.663979,0.398457,0.402233,0.538077,0.505173
1,0.0,0.057432,0.057239,0.080110,0.114412,0.073798,0.059603,0.088668
2,0.0,0.530405,0.354875,0.792315,0.413883,0.412125,0.468831,0.357116
3,0.0,0.628378,0.615251,0.982137,0.499567,0.602159,0.657190,0.540116
4,1.0,0.847973,0.527615,0.993841,0.544948,0.689416,0.702381,0.540880
5,0.0,0.070946,0.079734,0.287218,0.118313,0.173991,0.096967,0.113139
6,1.0,0.060811,0.066999,0.186049,0.158183,0.124462,0.076429,0.095498
7,1.0,0.483108,0.313623,0.889087,0.427832,0.519163,0.438963,0.332026
8,0.0,0.141892,0.077855,0.284754,0.181301,0.202901,0.124159,0.121485
9,0.0,0.334459,0.314537,0.856990,0.414141,0.347957,0.376751,0.302638


In [20]:
evaluate_ensemble(df_train, col_pred='stacking')

[auc] model1:0.8061,model2:0.8215,model3:0.7669,model4:0.7536,model5:0.7537  ->    emsamble]0.8220


In [22]:
sub = df_test.copy()    
sub['ensemble'] = np.where(sub['ensemble'] < 0.5, 0, 1)
sub = sub[['index', 'ensemble']] 
sub.to_csv('sub/submission_stack_w.csv', index=None, header=False,)